In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [7]:
edf = pd.read_parquet('../data/oneday/')
edf.head(3)

,rnum,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,...,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt,load_dt,multiMovieYn,repNationCd
0,1,1,0,OLD,20137048,국제시장,2014-12-17,6129982500,47.3,2797599300,...,47219550096,751754,275212,57.8,6097431,941,4650,20150101,N,NaN
1,2,2,1,OLD,20143642,테이큰 3,2015-01-01,2644551100,20.4,1640018100,...,3658460100,321653,177118,122.5,467280,614,2947,20150101,N,NaN
2,3,3,-1,OLD,20149859,마다가스카의 펭귄,2014-12-31,1687516200,13.0,672528400,...,2736013000,212779,67641,46.6,361669,594,1846,20150101,N,NaN


In [8]:
edf.to_parquet('../data/extract/spark/', partition_cols=['load_dt','multiMovieYn', 'repNationCd'])

In [3]:
kinddf = edf[['movieCd', 'multiMovieYn', 'repNationCd']].drop_duplicates()
kinddf.head(3)


,movieCd,multiMovieYn,repNationCd
0,20137048,N,NaN
1,20143642,N,NaN
2,20149859,N,NaN


In [4]:
len(kinddf)

1257

In [5]:
len(kinddf['movieCd'].drop_duplicates())

798

In [6]:
multi_nan = kinddf[kinddf.multiMovieYn.isna()]
multi_nan.head(3)

,movieCd,multiMovieYn,repNationCd
20,20143642,NaN,F
21,20149859,NaN,F
22,20140226,NaN,F


In [7]:
multi_nan[multi_nan.repNationCd.isna()].size

0

In [8]:
nation_nan = kinddf[kinddf.repNationCd.isna()]
nation_nan.head(3)

,movieCd,multiMovieYn,repNationCd
0,20137048,N,NaN
1,20143642,N,NaN
2,20149859,N,NaN


In [9]:
nation_nan[nation_nan.multiMovieYn.isna()].size

0

In [10]:
m_df = pd.merge(multi_nan, nation_nan, on=['movieCd'], suffixes=('_m', '_n'), how='outer')
m_df.head(3)

,movieCd,multiMovieYn_m,repNationCd_m,multiMovieYn_n,repNationCd_n
0,19288001,NaN,NaN,N,NaN
1,19358007,NaN,F,N,NaN
2,19380009,NaN,NaN,Y,NaN


In [11]:
m_df.size

3990

In [12]:
m_df.drop_duplicates().size

3990

### 검증

In [13]:
m_df[m_df['multiMovieYn_m'].notna()]

,movieCd,multiMovieYn_m,repNationCd_m,multiMovieYn_n,repNationCd_n


In [18]:
m_df[m_df['repNationCd_m'].isna()].size

880

In [19]:
m_df[m_df['multiMovieYn_n'].isna()].size

815

In [17]:
m_df[m_df['repNationCd_n'].notna()]

,movieCd,multiMovieYn_m,repNationCd_m,multiMovieYn_n,repNationCd_n


### 메타(국내, 독립) 데이터 추출

In [38]:
meta_df = m_df[['movieCd','repNationCd_m','multiMovieYn_n']]
meta_df.columns = ['movieCd','repNationCd','multiMovieYn']
meta_df.head(3)

,movieCd,repNationCd,multiMovieYn
0,20143642,F,N
1,20149859,F,N
2,20140226,F,N


In [42]:
# 검증
meta_df.isna().any(axis=1).sum()

np.int64(0)

In [46]:
if meta_df['movieCd'].duplicated().any():
    print("중복 있음")
else:
    print("중복 없음")
    

중복 없음


In [54]:
vc = meta_df['movieCd'].value_counts()
dup = vc[vc > 1]
dup.index.tolist()

[]

In [57]:
len(meta_df['movieCd'].drop_duplicates()) == len(meta_df)

True

# 누락 있음

In [58]:
len(kinddf['movieCd'].drop_duplicates()) == len(meta_df)

False